<a href="https://colab.research.google.com/github/mehedihasan-cse/ML-Projects/blob/main/Hate_Speech_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [734]:
import numpy as np  #To work with multi-dimentional array and matrix
import pandas as pd  #Data Manipulation, Data strcutre
import matplotlib.pyplot as plt #Data Visualization
import seaborn as sns
import nltk
import string
%matplotlib inline

In [735]:
df = pd.read_csv("/content/bengali_hate_v2.0.csv")

In [736]:
df.tail(7)

,text,label,target
5691,তুই তো নাস্তিক সেটা আমরা সবাই জানি তুই তো আরো ...,Religious,2
5692,সত্যি করে বল ডাইনোসর বিলুপ্ত হয়ে গেছিল নাকি তো...,Geopolitical,3
5693,জুতা যতই দামি হোক তার স্থান পায়ে তার স্থান কখ...,Personal,0
5694,"আমাদের দেশ ছোট বলে ভারতের চেয়ে পিছিয়ে, না হয়...",Geopolitical,3
5695,হিন্দুদের মধ্যে এগুলো হয় এটাই তার বাস্তব প্রমা...,Personal,0
5696,দেখলে মনে হয় শালী একটা অস্ট্রেলিয়ান গাভী,Personal,0
5697,ভাই আমি আপনার সাথে একমত। আর নারী নেতৃত্বে থাকত...,Political,1


In [737]:
df.columns.values

array(['text', 'label', 'target'], dtype=object)

In [738]:
df.value_counts()

,,,count
text,label,target,
অন্ন কেহু না সব রেন্ডিয়ার দাদা বাবুরা। চিন্তার কারন নাই।,Geopolitical,3,1
ভাই আপনি তাকে কেন বলছেন না যে তুমি উর্দুতে কথা বল আমি যেভাবে ধর্ম গ্রন্থ থেকে রেফারেন্স দিয়ে কথা বলতেছি তুমি রেফারেন্সে কথা বল এটা বলুন,Religious,2,1
ভাই তোর সাফল্যের রহস্যটা কি? আমি পর্ণ ওয়েবসাইট ব্যান করি নাই,Personal,0,1
ভাই তোর মতো মা চোদা আর দেখিনি,Personal,0,1
ভাই তোমারে তো চুরের মত লাগে.,Personal,0,1
...,...,...,...
কিছু কুচুটে লোকের অসৎ উদ্দেশ্য চরিতার্থ করার জন্য আপনি ভিডিও প্রচার করে যাওয়া থেকে বিরত থাকার অনুরোধ করছিভালো থাকুন,Personal,0,1
"কি সব বাল সাল, সোনিকা ধুর ক্ষ্যাত",Personal,0,1
"কি যে ওর কন্ঠ সুর শুনলে পাড়েই বমি আসে, হিমেশের সালমান খান বেশি ফেমাস হওয়ার জন্য এইসব পাগলদের কে প্রশ্রয় দেয়",Personal,0,1


# **Data Pre-Processing**

In [739]:
df.drop_duplicates(inplace =True)
df = df.replace('\n','',regex= True)
df = df.replace(',,',',',regex= True)

In [740]:
df

,text,label,target
0,বৌদির দুধ দেকে তো আমার ই চোখ ঠিক ছিলো না - পোল...,Personal,0
1,এই সরকার কে যারা নির্লজ্জের মত সাপোর্ট দিয়েছে ...,Political,1
2,পিলখানা হত্যাকান্ড বাংলাদেশের প্রতিরক্ষা ব্যবস...,Geopolitical,3
3,ভারতের অর্থনীতি নিয়ে আপনাদের ভাবতে হবে না। ভা...,Geopolitical,3
4,খানকির পুলা মালায়নদের মেরে সাফা করে ফেল,Personal,0
...,...,...,...
5693,জুতা যতই দামি হোক তার স্থান পায়ে তার স্থান কখ...,Personal,0
5694,"আমাদের দেশ ছোট বলে ভারতের চেয়ে পিছিয়ে, না হয়...",Geopolitical,3
5695,হিন্দুদের মধ্যে এগুলো হয় এটাই তার বাস্তব প্রমা...,Personal,0
5696,দেখলে মনে হয় শালী একটা অস্ট্রেলিয়ান গাভী,Personal,0


In [741]:
punct=string.punctuation

In [742]:
def remove_punct(x):
    if isinstance(x, str):  # Check if x is a string
        return x.translate(str.maketrans("", "", punct))
    else:
        return x  # Return the original value if it's not a string

df['text'] = df['text'].apply(lambda x: remove_punct(x))

In [743]:
df

,text,label,target
0,বৌদির দুধ দেকে তো আমার ই চোখ ঠিক ছিলো না পোলা...,Personal,0
1,এই সরকার কে যারা নির্লজ্জের মত সাপোর্ট দিয়েছে ...,Political,1
2,পিলখানা হত্যাকান্ড বাংলাদেশের প্রতিরক্ষা ব্যবস...,Geopolitical,3
3,ভারতের অর্থনীতি নিয়ে আপনাদের ভাবতে হবে না। ভা...,Geopolitical,3
4,খানকির পুলা মালায়নদের মেরে সাফা করে ফেল,Personal,0
...,...,...,...
5693,জুতা যতই দামি হোক তার স্থান পায়ে তার স্থান কখ...,Personal,0
5694,আমাদের দেশ ছোট বলে ভারতের চেয়ে পিছিয়ে না হয় ...,Geopolitical,3
5695,হিন্দুদের মধ্যে এগুলো হয় এটাই তার বাস্তব প্রমা...,Personal,0
5696,দেখলে মনে হয় শালী একটা অস্ট্রেলিয়ান গাভী,Personal,0


In [744]:
df.isna().sum()

,0
text,0
label,0
target,0


In [745]:
df = df.fillna(method= 'pad')

In [746]:
df.isna().sum()

,0
text,0
label,0
target,0


In [747]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5698 entries, 0 to 5697
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5698 non-null   object
 1   label   5698 non-null   object
 2   target  5698 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 133.7+ KB


In [748]:
nltk.download('stopwords')
from nltk.corpus import stopwords
",".join(stopwords.words('bengali'))
stop_words=set(stopwords.words('bengali'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [749]:
" ,".join(stopwords.words('bengali'))

'অতএব ,অথচ ,অথবা ,অনুযায়ী ,অনেক ,অনেকে ,অনেকেই ,অন্তত ,অন্য ,অবধি ,অবশ্য ,অর্থাত ,আই ,আগামী ,আগে ,আগেই ,আছে ,আজ ,আদ্যভাগে ,আপনার ,আপনি ,আবার ,আমরা ,আমাকে ,আমাদের ,আমার ,আমি ,আর ,আরও ,ই ,ইত্যাদি ,ইহা ,উচিত ,উত্তর ,উনি ,উপর ,উপরে ,এ ,এঁদের ,এঁরা ,এই ,একই ,একটি ,একবার ,একে ,এক্ ,এখন ,এখনও ,এখানে ,এখানেই ,এটা ,এটাই ,এটি ,এত ,এতটাই ,এতে ,এদের ,এব ,এবং ,এবার ,এমন ,এমনকী ,এমনি ,এর ,এরা ,এল ,এস ,এসে ,ঐ ,ও ,ওঁদের ,ওঁর ,ওঁরা ,ওই ,ওকে ,ওখানে ,ওদের ,ওর ,ওরা ,কখনও ,কত ,কবে ,কমনে ,কয়েক ,কয়েকটি ,করছে ,করছেন ,করতে ,করবে ,করবেন ,করলে ,করলেন ,করা ,করাই ,করায় ,করার ,করি ,করিতে ,করিয়া ,করিয়ে ,করে ,করেই ,করেছিলেন ,করেছে ,করেছেন ,করেন ,কাউকে ,কাছ ,কাছে ,কাজ ,কাজে ,কারও ,কারণ ,কি ,কিংবা ,কিছু ,কিছুই ,কিন্তু ,কী ,কে ,কেউ ,কেউই ,কেখা ,কেন ,কোটি ,কোন ,কোনও ,কোনো ,ক্ষেত্রে ,কয়েক ,খুব ,গিয়ে ,গিয়েছে ,গিয়ে ,গুলি ,গেছে ,গেল ,গেলে ,গোটা ,চলে ,চান ,চায় ,চার ,চালু ,চেয়ে ,চেষ্টা ,ছাড়া ,ছাড়াও ,ছিল ,ছিলেন ,জন ,জনকে ,জনের ,জন্য ,জন্যওজে ,জানতে ,জানা ,জানানো ,জানায় ,জানিয়ে ,জানিয়েছে ,জে ,জ্নজন ,টি ,ঠিক ,তখন ,

In [750]:
def remove_stop(x):
  return " ".join([word for word in str(x).split()if word not in stop_words])
df['text']=df['text'].apply(lambda x : remove_stop(x))

In [751]:
df

,text,label,target
0,বৌদির দুধ দেকে চোখ ছিলো পোলাপান চোখ কিভাবে,Personal,0
1,সরকার নির্লজ্জের মত সাপোর্ট দিয়েছে বছরের বছর আ...,Political,1
2,পিলখানা হত্যাকান্ড বাংলাদেশের প্রতিরক্ষা ব্যবস...,Geopolitical,3
3,ভারতের অর্থনীতি আপনাদের ভাবতে না। ভারতের অর্থন...,Geopolitical,3
4,খানকির পুলা মালায়নদের মেরে সাফা ফেল,Personal,0
...,...,...,...
5693,জুতা যতই দামি স্থান পায়ে স্থান কখনো মাথায় হয...,Personal,0
5694,দেশ ছোট ভারতের পিছিয়ে মুতার জন্যও তোদের সাথে স...,Geopolitical,3
5695,হিন্দুদের এগুলো বাস্তব প্রমাণ সবাই মতামত,Personal,0
5696,দেখলে শালী একটা অস্ট্রেলিয়ান গাভী,Personal,0


In [752]:
from collections import Counter
cnt= Counter()

for text in df['text'].values:
  for word in text.split():
    cnt[word]+=1
cnt.most_common(2)

[('ভারত', 546), ('।', 503)]

In [753]:
freq_words= set([w  for (w,wc) in cnt.most_common(2)])

In [754]:
freq_words

{'।', 'ভারত'}

In [755]:
def remove_freq(x):
    return " ".join ([word for word in str(x).split() if word not in freq_words])
df['text']=df['text'].apply(lambda x: remove_freq(x))

In [756]:

#Drop column --> axis= 1 ; row --> axis= 0
#x = df.drop(columns = ['target','label'], axis= 1)
#y = df[['target','label']]
x= df.text.values
y= df.target.values

In [757]:
print(x)

['বৌদির দুধ দেকে চোখ ছিলো পোলাপান চোখ কিভাবে'
 'সরকার নির্লজ্জের মত সাপোর্ট দিয়েছে বছরের বছর আরো রাস্তায় রাস্তায় কাঁদতে'
 'পিলখানা হত্যাকান্ড বাংলাদেশের প্রতিরক্ষা ব্যবস্থা ধ্বংসের ভারতের প্রত্যক্ষ সহযোগিতায় হত্যাকা ঘটানো হয়েছিল'
 ... 'হিন্দুদের এগুলো বাস্তব প্রমাণ সবাই মতামত'
 'দেখলে শালী একটা অস্ট্রেলিয়ান গাভী'
 'ভাই সাথে একমত। নারী নেতৃত্বে থাকতে চাই না। ভিপি নুর মত সৎ সাহসী একজন নেতা চাই।']


In [758]:
print(y)

[0 1 3 ... 0 0 1]


In [759]:
from sklearn.model_selection import train_test_split

In [760]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2)

In [761]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [762]:
cv= CountVectorizer()
x_train = cv.fit_transform(xtrain)
x_test = cv.transform(xtest)

## **Model Algorithm**

In [763]:
model = MultinomialNB()
model.fit(x_train,ytrain)

MultinomialNB()

In [764]:
labels = model.score(x_test, ytest)
print(labels)

0.5903508771929824


In [765]:
model = LogisticRegression()
model.fit(x_train, ytrain)

LogisticRegression()

In [766]:
labels = model.score(x_test, ytest)
print(labels)

0.6368421052631579


**TF-IDF**

In [767]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [768]:
vectorizer = TfidfVectorizer()

x_train = vectorizer.fit_transform(xtrain)
x_test = vectorizer.transform(xtest)

In [769]:
model = MultinomialNB()
model.fit(x_train,ytrain)

MultinomialNB()

In [770]:
labels = model.score(x_test, ytest)
print(labels)

0.5535087719298246


In [771]:
from sklearn.linear_model import LogisticRegression

In [772]:
model = LogisticRegression()
model.fit(x_train, ytrain)

LogisticRegression()

In [773]:
labels = model.score(x_test, ytest)
print(labels)

0.6201754385964913


# **Draft Code**